## Set environment

In [1]:
%cd data_generation

[Errno 2] No such file or directory: 'data_generation'
/mnt/d/deepface-video-recognition/data_generation


/home/huy/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


## Import lib

In [2]:
import cv2
import os
import concurrent.futures
import queue
import threading
import pickle
import matplotlib.pyplot as plt

from deepface import DeepFace

2024-08-13 13:38:32.263511: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 13:38:32.263588: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 13:38:32.263624: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 13:38:34.043427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Detect Faces

In [3]:
movie = "like_me"
# det_faces = ['fastmtcnn', 'opencv', 'retinaface']
# movie_det = (
    # (f"{movie}", "fastmtcnn"),
    # (f"{movie}", "opencv"),
    # (f"{movie}", "retinaface"),
    # ("Calloused_Hands", "fastmtcnn"),
    # ("Calloused_Hands", "opencv"),
    # ("Calloused_Hands", "retinaface"),
    # ("Memphis", "fastmtcnn"),
    # ("Memphis", "opencv"),
    # ("Memphis", "retinaface"),
    # ("Liberty_Kid", "fastmtcnn"),
    # ("Liberty_Kid", "opencv"),
    # ("Liberty_Kid", "retinaface"),
    # ("like_me", "fastmtcnn"),
    # ("like_me", "opencv"),
    # ("like_me", "retinaface"),
    # ("losing_ground", "fastmtcnn"),
    # ("losing_ground", "opencv"),
    # ("losing_ground", "retinaface"),
# )
workers = 5
frame_source_folder = "frames"
faces_des_folder = "det_faces"


In [5]:
def extract_faces_parallel(img, des_file_path, backend):
    extract_face_file_path = os.path.splitext(des_file_path)[0] + f"-det_{backend}.pkl"
    if os.path.exists(extract_face_file_path):
        # os.remove(extract_face_file_path)
        # print(f"Deleted face detection: {extract_face_file_path}")
        print(f"Face detection already exists: {extract_face_file_path}")
        return
    # Detect faces using DeepFace
    extract_face_deepface = DeepFace.extract_faces(img, detector_backend=backend, enforce_detection=False)
    extraction_face_data = []
    img_height, img_width = img.shape[:2]
    for extraction in extract_face_deepface:  
        if extraction['confidence'] >= 0.90:
            # extraction_face_data.append(extraction['facial_area'])
            face = extraction['facial_area']
            x = max(face['x'], 0)
            y = max(face['y'], 0)
            w = min(face['w'], img_width - x)
            h = min(face['h'], img_height - y)
            # Crop the face region
            cropped_face = img[y:y+h, x:x+w]
            extraction_face_data.append(cropped_face)

    with open(extract_face_file_path, 'wb') as f:
        pickle.dump(extraction_face_data, f)
        print(f"Created face detection: {extract_face_file_path}")

In [6]:
def detect_face_from_img(movie, backend):
    global frame_source_folder, faces_des_folder
    imgs_source_dir = f"../{frame_source_folder}/{movie}/"
    faces_des_dir = f"../{faces_des_folder}/{movie}/"
    # Traverse all files and subfolders in the source folder
    for root, _, files in os.walk(imgs_source_dir):
        # Create a ThreadPoolExecutor with the number of CPU cores
        with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
            for file in files:
                # Construct source file path
                source_file_path = os.path.join(root, file)

                # Construct destination file path
                des_file_path = os.path.join(faces_des_dir, os.path.relpath(source_file_path, imgs_source_dir))
                des_dir = os.path.dirname(des_file_path)
                os.makedirs(des_dir, exist_ok=True)
                if source_file_path.lower().endswith('.jpg'):
                    img = cv2.imread(source_file_path)

                    # Use ThreadPoolExecutor to parallelize the face detection task
                    executor.submit(extract_faces_parallel, img, des_file_path, backend)


In [ ]:
detect_face_from_img(movie, "fastmtcnn")

In [ ]:
detect_face_from_img(movie, "opencv")

In [ ]:
detect_face_from_img(movie, "retinaface")

In [ ]:
print("Done")

In [ ]:
# def print_incresement(movie, backend):
#     print(f"Movie: {movie}, Backend: {backend}")
# push_task_to_queue([(print_incresement, (movie, backend))for (movie, backend) in movie_det])